# Capstone Project

### Week Three: Exploring and clustering the neighborhoods in Toronto

# Question 1

### 1. Import the Libraries

In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np
print('Pandas and numpy imported successfully')

Pandas and numpy imported successfully


In [2]:
# Lets install lxml required for the next step
!conda install -c anaconda lxml --yes
print(" Extra libraries installed")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    libxslt-1.1.33             |       h7d1a2b0_0         577 KB  anaconda
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.2 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               anaconda/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages wi

### 2. Import the table

In [227]:
# lets use the pandas get_html() to view the tables in wikipedia site
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Assign the first table to a pandas Dataframe
tdf = tables[0]
# tdf.head()

# Let us see how many Boroughs are not assigned
print(tdf['Borough'].value_counts())

#  Let us get the index of rows with a Not assigned Borough
my_index = tdf[tdf['Borough'] == "Not assigned"].index

# Let us drop rows with corresponding index above
tdf.drop(my_index, inplace=True)
tdf.reset_index(drop=True)

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Get shape of the dataframe

In [146]:
tdf.shape

(103, 3)

# Question 2

In [215]:
# Get the location file into a dataframe
ldf = pd.read_csv("http://cocl.us/Geospatial_data")
ldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [234]:
# Merge dataframes on Postal code
my_tdf = pd.merge(tdf,ldf, on="Postal Code")
my_tdf.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Question 3

In [235]:
from geopy.geocoders import Nominatim
import folium

In [239]:
# Working with Boroughs in Toronto
tr_df = my_tdf[my_tdf['Borough'].str.contains('Toronto')].reset_index(drop=True)
tr_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [240]:
# Get the location data for Toronto
address = "Toronto, CA"
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [261]:
# Create map of Toronto
toronto_map = folium.Map(location=[latitude,longitude], zoom_start=12)

# Use for loop to populate map with our Toronto dataframe data
for lat,lng,borough,neigh in zip(tr_df['Latitude'],tr_df['Longitude'],tr_df['Borough'],tr_df['Neighborhood']):
    label = '{}, {}'.format(neigh,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(toronto_map)
toronto_map